In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 7.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

from scipy.stats import norm
import numpy as np
import tensorflow as tf

In [3]:
url = "https://raw.githubusercontent.com/Tejanikhil/Shared-ASAG/main"
df = pd.read_csv(url + "/SemEvalData.csv")

In [4]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
def extract_pos(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    pos_words = [word for word, pos in pos_tags if pos.startswith('VB') or pos.startswith('JJ') or pos.startswith('NN')]
    return ' '.join(pos_words)

def process_dataset(text):
    lower_cased = text.lower()
    tokens = nltk.word_tokenize(lower_cased)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    preprocessed = ' '.join(filtered_tokens)
    return preprocessed

In [6]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(df.loc[:,["Actual Answer","Student Answer"]], df["Grade"], test_size=0.1, random_state=42)

In [8]:
TRAIN = train_X
TRAIN["LABELS"] = train_y
TEST = test_X
TEST["LABELS"] = test_y

In [9]:
train_X["Actual Answer"] = train_X["Actual Answer"].apply(extract_pos).apply(process_dataset)
train_X["Student Answer"] = train_X["Student Answer"].apply(extract_pos).apply(process_dataset)

test_X["Actual Answer"] = test_X["Actual Answer"].apply(extract_pos).apply(process_dataset)
test_X["Student Answer"] = test_X["Student Answer"].apply(extract_pos).apply(process_dataset)

In [10]:
print(TRAIN.shape)
TRAIN = TRAIN.replace("", pd.NA).dropna()
print(TRAIN.shape)
TEST = TEST.replace("", pd.NA).dropna()

(2197, 3)
(2184, 3)


In [11]:
train_X = TRAIN.drop("LABELS", axis = 1)
train_y = TRAIN.iloc[:,-1]
test_X = TEST.drop("LABELS", axis = 1)
test_y = TEST.iloc[:,-1]

In [12]:
sentences = list(train_X["Actual Answer"]) + list(train_X["Student Answer"])
sentences = [i.split() for i in sentences]

In [13]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [14]:
train_Embeddings_df = pd.DataFrame({"Student_Answer_Embeddings":train_X["Student Answer"], "Reference_Answer_Embeddings":train_X["Actual Answer"]})
test_Embeddings_df = pd.DataFrame({"Student_Answer_Embeddings":test_X["Student Answer"], "Reference_Answer_Embeddings":test_X["Actual Answer"]})

In [15]:
model = Word2Vec(sentences, min_count=1)

In [16]:
def get_wordvec_embeddings(text):
    try:
        embeds = [model.wv[i] for i in text.split()]
    except:
        embeds = np.zeros(100)
        
    return np.mean(np.array(embeds),axis = 0)

In [17]:
train_Embeddings_df["Student_Answer_Embeddings"] = train_Embeddings_df["Student_Answer_Embeddings"].apply(get_wordvec_embeddings)
train_Embeddings_df["Reference_Answer_Embeddings"] = train_Embeddings_df["Reference_Answer_Embeddings"].apply(get_wordvec_embeddings)

test_Embeddings_df["Student_Answer_Embeddings"] = test_Embeddings_df["Student_Answer_Embeddings"].apply(get_wordvec_embeddings)
test_Embeddings_df["Reference_Answer_Embeddings"] = test_Embeddings_df["Reference_Answer_Embeddings"].apply(get_wordvec_embeddings)

In [18]:
train_Embeddings_df

,Student_Answer_Embeddings,Reference_Answer_Embeddings
199,"[-0.013231093, 0.01926258, -0.03497585, 0.0240...","[-0.11933394, 0.07470847, -0.18360311, 0.14704..."
229,"[-0.0351708, 0.2985341, -0.64064443, 0.3386988...","[-0.031117108, 0.09332319, -0.5197473, 0.32609..."
521,"[-0.053558454, 0.08091264, -0.22860163, 0.1696...","[-0.19110733, 0.062203974, -0.24957447, 0.2336..."
651,"[-0.06562183, 0.22335838, -0.20084943, 0.12248...","[-0.056014687, 0.12857561, -0.16960359, 0.1367..."
450,"[-0.058086336, 0.049783032, -0.23313145, 0.169...","[-0.0353079, 0.07401244, -0.33640778, 0.262426..."
...,...,...
1638,"[-0.09139938, 0.09371384, -0.057737358, 0.0883...","[-0.1676393, 0.2461407, 0.31873792, -0.0960089..."
1095,"[0.014017844, 0.768235, -0.6110421, 0.22033174...","[-0.014550311, 0.11825945, -0.36267385, 0.3493..."
1130,"[-0.010181204, 0.26435462, -0.42537203, 0.3596...","[-0.0894115, 0.034320932, -0.02979892, 0.04476..."
1294,"[-0.06864554, 0.10291038, -0.1652634, 0.173739...","[-0.07753556, 0.09434413, -0.12873547, 0.11807..."


In [19]:
train_X["Subtracted_Embeddings"] = train_Embeddings_df.apply(lambda x:x["Student_Answer_Embeddings"]-x["Reference_Answer_Embeddings"], axis = 1)
test_X["Subtracted_Embeddings"] = test_Embeddings_df.apply(lambda x:x["Student_Answer_Embeddings"]-x["Reference_Answer_Embeddings"], axis = 1)

In [20]:
train_df = pd.DataFrame(list(train_X["Subtracted_Embeddings"]))
test_df = pd.DataFrame(list(test_X["Subtracted_Embeddings"]))

In [21]:
train_df.to_csv("Word2Vec_train_Embeddings.csv")
test_df.to_csv("Word2Vec_test_Embeddings.csv")

In [22]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()
LR_model.fit(train_df, train_y)

LinearRegression()

In [23]:
preds = LR_model.predict(test_df)
y_preds = [round(y_val, 1) for y_val in preds]
actuals = test_y
y_true = [round(y_val, 1) for y_val in actuals]

In [24]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
LR_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
LR_mse = mean_squared_error(y_true, np.round(y_preds, 1))
LR_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
LR_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
LR_r2 = r2_score(y_true, y_preds)

In [25]:
print("Mean Absolute Error (MAE): ", np.round(LR_mae, 2))
print("Mean Squared Error (MSE): ", np.round(LR_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(LR_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(LR_mape, 2))
print("R2 Score: ", np.round(LR_r2, 2))

Mean Absolute Error (MAE):  1.12
Mean Squared Error (MSE):  1.85
Root Mean Squared Error (RMSE):  1.36
Mean Absolute Percentage Error (MAPE):  468742008606369.25
R2 Score:  0.08


In [26]:
from sklearn import neighbors
rmse_val = []
mae_val = []
mse_val = []
r2_val = []
for K in range(2,20):
    knn_model = neighbors.KNeighborsRegressor(n_neighbors = K)
    knn_model.fit(train_df, train_y)  #fit the model
    pred=knn_model.predict(test_df) #make prediction on test set
    
    mae = mean_absolute_error(y_true, np.round(pred, 1))
    mae_val.append(mae)
    
    mse = mean_squared_error(y_true, np.round(pred, 1))
    mse_val.append(mse)
    
    rmse = mean_squared_error(y_true, np.round(pred, 1), squared=False)
    rmse_val.append(rmse)
    
    r2 = r2_score(y_true, pred)
    r2_val.append(r2)

In [27]:
y_true = test_y
knn_preds = knn_model.predict(test_df)

In [28]:
min_val = min(r2_val)
index = r2_val.index(min_val)
knn_r2 = min_val
knn_rmse = rmse_val[index]
knn_mae = mae_val[index]
knn_mse = mse_val[index]

In [29]:
print("Mean Absolute Error (MAE): ", np.round(knn_mae, 2))
print("Mean Squared Error (MSE): ", np.round(knn_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(knn_rmse, 2))
print("R2 Score: ", np.round(knn_r2, 2))

Mean Absolute Error (MAE):  1.22
Mean Squared Error (MSE):  2.84
Root Mean Squared Error (RMSE):  1.69
R2 Score:  -0.41


In [30]:
ada_reg = AdaBoostRegressor(n_estimators = 100, learning_rate=0.5)

ada_reg.fit(train_df, train_y)

ada_reg_pred = ada_reg.predict(pd.DataFrame(test_df))
ada_reg_mae = mean_absolute_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mse = mean_squared_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mape = mean_absolute_percentage_error(test_y, np.round(ada_reg_pred,1), multioutput = "uniform_average")
ada_reg_rmse = mean_squared_error(test_y, np.round(ada_reg_pred, 1), squared=False)
ada_reg_r2 = r2_score(test_y, ada_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(ada_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(ada_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(ada_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(ada_reg_mape, 2))
print("R2 Score: ", np.round(ada_reg_r2, 2))

Mean Absolute Error (MAE):  1.31
Mean Squared Error (MSE):  2.22
Root Mean Squared Error (RMSE):  1.49
Mean Absolute Percentage Error (MAPE):  538593751354920.9
R2 Score:  -0.12


In [31]:
gb_reg = GradientBoostingRegressor(n_estimators = 50, max_depth=5)

gb_reg.fit(train_df, train_y)

gb_reg_pred = gb_reg.predict(pd.DataFrame(test_df))
gb_reg_mae = mean_absolute_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mse = mean_squared_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mape = mean_absolute_percentage_error(test_y, np.round(gb_reg_pred,1), multioutput = "uniform_average")
gb_reg_rmse = mean_squared_error(test_y, np.round(gb_reg_pred, 1), squared=False)
gb_reg_r2 = r2_score(test_y, gb_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(gb_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(gb_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(gb_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(gb_reg_mape, 2))
print("R2 Score: ", np.round(gb_reg_r2, 2))

Mean Absolute Error (MAE):  1.06
Mean Squared Error (MSE):  1.67
Root Mean Squared Error (RMSE):  1.29
Mean Absolute Percentage Error (MAPE):  435654331300738.06
R2 Score:  0.15


In [32]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', seed = 123)

xg_reg.fit(train_df, train_y)

# print("Best parameters: ", xg_boost_grid_search.best_params_)
# print("Best score: ", xg_boost_grid_search.best_score_)
xg_reg_pred = xg_reg.predict(pd.DataFrame(test_df))
xg_reg_mae = mean_absolute_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mse = mean_squared_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mape = mean_absolute_percentage_error(test_y, np.round(xg_reg_pred,1), multioutput = "uniform_average")
xg_reg_rmse = mean_squared_error(test_y, np.round(xg_reg_pred, 1), squared=False)
xg_reg_r2 = r2_score(test_y, xg_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(xg_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(xg_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(xg_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(xg_reg_mape, 2))
print("R2 Score: ", np.round(xg_reg_r2, 2))

Mean Absolute Error (MAE):  1.13
Mean Squared Error (MSE):  2.13
Root Mean Squared Error (RMSE):  1.46
Mean Absolute Percentage Error (MAPE):  395213844850880.5
R2 Score:  -0.08


In [ ]:
32